# 🗺️ US Counties Analytics Dashboard

Interactive regional statistics with real US counties data and clean, modern design.

In [ ]:
# Setup and imports
import pandas as pd
import geopandas as gpd
import folium
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import requests
import zipfile
import os
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
from shapely.geometry import Polygon
import warnings
warnings.filterwarnings('ignore')

print('🚀 Setting up US Counties Dashboard...')

In [ ]:
class SimpleCountiesDashboard:
    """Simple, working counties dashboard"""
    
    def __init__(self):
        self.gdf = None
        self.stats_data = None
        
        # Clean colors
        self.colors = {
            'primary': '#2D3748',
            'surface': '#FFFFFF', 
            'text': '#1A202C',
            'text_light': '#718096',
            'border': '#E2E8F0',
            'background': '#F7FAFC'
        }
        
        # Outputs
        self.header_output = widgets.Output()
        self.controls_output = widgets.Output()
        self.stats_output = widgets.Output()
        self.map_output = widgets.Output()
    
    def download_counties_data(self):
        """Download real US counties or create fallback"""
        print('📥 Attempting to download US Counties data...')
        
        try:
            # Try to download real counties data
            url = "https://www2.census.gov/geo/tiger/GENZ2020/shp/cb_2020_us_county_20m.zip"
            filename = "counties.zip"
            
            if not os.path.exists(filename):
                print('  Downloading from Census Bureau...')
                response = requests.get(url, timeout=30)
                with open(filename, 'wb') as f:
                    f.write(response.content)
                print(f'  ✅ Downloaded {len(response.content)/1024/1024:.1f}MB')
            
            # Extract
            extract_dir = 'counties_data'
            if not os.path.exists(extract_dir):
                with zipfile.ZipFile(filename, 'r') as zip_ref:
                    zip_ref.extractall(extract_dir)
                print('  ✅ Extracted shapefile')
            
            # Load shapefile
            shp_files = [f for f in os.listdir(extract_dir) if f.endswith('.shp')]
            if shp_files:
                shp_path = os.path.join(extract_dir, shp_files[0])
                full_gdf = gpd.read_file(shp_path)
                
                # Filter to major states for performance
                major_states = ['06', '48', '12', '36', '17', '42', '08', '04', '13', '37']
                self.gdf = full_gdf[full_gdf['STATEFP'].isin(major_states)].copy()
                self.gdf = self.gdf.to_crs(epsg=4326)
                
                print(f'  ✅ Loaded {len(self.gdf)} counties from major states')
                return True
                
        except Exception as e:
            print(f'  ⚠️ Download failed: {e}')
        
        # Fallback to sample data
        print('  🔄 Using sample counties data...')
        return self.create_sample_counties()
    
    def create_sample_counties(self):
        """Create sample county boundaries"""
        
        # Major US counties with realistic coordinates
        counties_data = [
            # California
            ('Los Angeles County', 'CA', '06', (-118.9, 33.7, -117.6, 34.8)),
            ('San Diego County', 'CA', '06', (-117.6, 32.5, -116.1, 33.5)),
            ('Orange County', 'CA', '06', (-118.1, 33.3, -117.4, 33.9)),
            ('Riverside County', 'CA', '06', (-117.7, 33.4, -114.1, 34.1)),
            ('San Bernardino County', 'CA', '06', (-118.0, 34.0, -114.1, 35.8)),
            
            # Texas
            ('Harris County', 'TX', '48', (-95.8, 29.5, -94.9, 30.1)),
            ('Dallas County', 'TX', '48', (-97.0, 32.6, -96.4, 33.0)),
            ('Tarrant County', 'TX', '48', (-97.5, 32.6, -96.9, 33.0)),
            ('Bexar County', 'TX', '48', (-98.8, 29.1, -98.0, 29.8)),
            ('Travis County', 'TX', '48', (-98.2, 30.0, -97.5, 30.5)),
            
            # Florida  
            ('Miami-Dade County', 'FL', '12', (-80.9, 25.1, -80.1, 25.9)),
            ('Broward County', 'FL', '12', (-80.6, 25.9, -80.0, 26.4)),
            ('Palm Beach County', 'FL', '12', (-80.6, 26.3, -79.8, 27.0)),
            ('Hillsborough County', 'FL', '12', (-82.8, 27.6, -82.0, 28.2)),
            ('Orange County', 'FL', '12', (-81.7, 28.4, -81.1, 28.8)),
            
            # New York
            ('New York County', 'NY', '36', (-74.1, 40.7, -73.9, 40.8)),
            ('Kings County', 'NY', '36', (-74.1, 40.5, -73.8, 40.7)),
            ('Queens County', 'NY', '36', (-73.9, 40.5, -73.7, 40.8)),
            ('Suffolk County', 'NY', '36', (-73.3, 40.6, -71.9, 41.2)),
            ('Nassau County', 'NY', '36', (-73.8, 40.6, -73.3, 40.8)),
        ]
        
        geometries = []
        names = []
        states = []
        statefps = []
        
        for name, state, statefp, (min_lon, min_lat, max_lon, max_lat) in counties_data:
            coords = [
                (min_lon, min_lat),
                (max_lon, min_lat),
                (max_lon, max_lat), 
                (min_lon, max_lat),
                (min_lon, min_lat)
            ]
            
            geometries.append(Polygon(coords))
            names.append(name)
            states.append(state)
            statefps.append(statefp)
        
        self.gdf = gpd.GeoDataFrame({
            'NAME': names,
            'STUSPS': states,
            'STATEFP': statefps,
            'geometry': geometries
        })
        
        print(f'  ✅ Created {len(self.gdf)} sample counties')
        return True
    
    def generate_county_statistics(self):
        """Generate comprehensive synthetic statistics"""
        print('📊 Generating synthetic county statistics...')
        
        counties = self.gdf['NAME'].tolist()
        states = self.gdf['STUSPS'].tolist()
        
        # Generate 18 months of data
        dates = pd.date_range(end=pd.Timestamp.now(), periods=18, freq='M')
        
        data_rows = []
        
        for county, state in zip(counties, states):
            # Set realistic base values by county size/type
            if 'Los Angeles' in county:
                profile = {'pop': 10_000_000, 'gdp_pc': 70000, 'unemp_base': 5.5, 'type': 'major_metro'}
            elif any(x in county for x in ['New York', 'Kings', 'Queens']):
                profile = {'pop': 2_500_000, 'gdp_pc': 75000, 'unemp_base': 4.8, 'type': 'major_metro'}
            elif any(x in county for x in ['Harris', 'Dallas', 'Cook', 'Miami-Dade']):
                profile = {'pop': 4_000_000, 'gdp_pc': 65000, 'unemp_base': 5.2, 'type': 'metro'}
            elif 'County' in county:
                profile = {'pop': np.random.randint(500_000, 2_000_000), 'gdp_pc': np.random.uniform(50000, 70000), 
                          'unemp_base': np.random.uniform(4.5, 7.5), 'type': 'suburban'}
            else:
                profile = {'pop': np.random.randint(200_000, 1_000_000), 'gdp_pc': np.random.uniform(45000, 65000),
                          'unemp_base': np.random.uniform(5.0, 8.0), 'type': 'small'}
            
            for i, date in enumerate(dates):
                # Add realistic growth and variation
                months_from_start = i
                growth_factor = (1.01) ** (months_from_start / 12)  # 1% annual growth
                seasonal_factor = 1 + 0.02 * np.sin(2 * np.pi * date.month / 12)  # 2% seasonal variation
                economic_cycle = 1 + 0.05 * np.sin(2 * np.pi * months_from_start / 48)  # 4-year cycle
                
                current_pop = int(profile['pop'] * growth_factor * seasonal_factor)
                current_gdp_pc = profile['gdp_pc'] * economic_cycle
                
                data_rows.append({
                    'county': county,
                    'state': state,
                    'date': date,
                    'county_type': profile['type'],
                    
                    # Core demographics
                    'population': current_pop,
                    'population_density': current_pop / np.random.uniform(100, 3000),
                    'median_age': np.random.uniform(32, 45),
                    'households': int(current_pop / np.random.uniform(2.2, 2.8)),
                    
                    # Age distribution
                    'age_under_18': int(current_pop * np.random.uniform(0.18, 0.28)),
                    'age_18_34': int(current_pop * np.random.uniform(0.20, 0.30)),
                    'age_35_54': int(current_pop * np.random.uniform(0.25, 0.32)),
                    'age_55_64': int(current_pop * np.random.uniform(0.12, 0.18)),
                    'age_65_plus': int(current_pop * np.random.uniform(0.12, 0.22)),
                    
                    # Economics
                    'gdp_millions': (current_pop * current_gdp_pc) / 1_000_000,
                    'gdp_per_capita': current_gdp_pc,
                    'median_income': current_gdp_pc * 0.75 * np.random.uniform(0.9, 1.1),
                    'unemployment_rate': max(2.0, profile['unemp_base'] * (2.2 - economic_cycle) + np.random.uniform(-0.8, 0.8)),
                    'poverty_rate': np.random.uniform(8, 20) * (2.5 - economic_cycle),
                    'labor_force_participation': np.random.uniform(60, 75),
                    
                    # Housing
                    'housing_units': int(current_pop / np.random.uniform(2.0, 2.6)),
                    'median_home_value': current_gdp_pc * np.random.uniform(4, 9) * economic_cycle,
                    'rent_burden_pct': np.random.uniform(25, 40),
                    
                    # Education
                    'high_school_grad_rate': np.random.uniform(80, 96),
                    'college_grad_rate': np.random.uniform(20, 50),
                    'education_score': np.random.uniform(65, 92),
                    
                    # Health & Safety
                    'life_expectancy': np.random.uniform(76, 82),
                    'crime_rate_per_100k': np.random.uniform(150, 650),
                    'violent_crime_rate': np.random.uniform(100, 400),
                    
                    # Infrastructure
                    'broadband_access_pct': np.random.uniform(70, 95),
                    'avg_commute_minutes': np.random.uniform(20, 40),
                    'air_quality_index': np.random.uniform(35, 85),
                    
                    # Migration
                    'net_migration': np.random.randint(-1000, 3000),
                    'domestic_migration': np.random.randint(-500, 2000),
                    'international_migration': np.random.randint(-200, 1500),
                })
        
        self.stats_data = pd.DataFrame(data_rows)
        
        print(f'  ✅ Generated {len(self.stats_data):,} statistical records')
        print(f'  📅 Time span: {dates[0].strftime("%Y-%m")} to {dates[-1].strftime("%Y-%m")}')
        print(f'  🏘️ Counties: {len(counties)}')
        print(f'  📊 Metrics: {len([col for col in self.stats_data.columns if col not in ["county", "state", "date", "county_type"]])}')
        
        return self.stats_data
    
    def create_dashboard_header(self):
        """Create clean dashboard header"""
        with self.header_output:
            header_html = f'''
            <div style="
                background: {self.colors['surface']};
                padding: 50px 20px;
                margin-bottom: 40px;
                border-bottom: 1px solid {self.colors['border']};
                text-align: center;
            ">
                <h1 style="
                    font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', sans-serif;
                    font-size: 2.8rem;
                    font-weight: 300;
                    color: {self.colors['text']};
                    margin: 0 0 16px 0;
                    letter-spacing: -0.02em;
                ">US Counties Analytics</h1>
                <p style="
                    font-size: 1.2rem;
                    color: {self.colors['text_light']};
                    margin: 0 0 12px 0;
                    font-weight: 400;
                ">Interactive demographic and economic data visualization</p>
                <p style="
                    font-size: 0.95rem;
                    color: {self.colors['text_light']};
                    margin: 0;
                ">🏘️ {len(self.gdf)} counties • 📅 18 months • 📈 25+ metrics</p>
            </div>
            '''
            display(HTML(header_html))
    
    def create_interactive_controls(self):
        """Create dashboard controls"""
        with self.controls_output:
            # Get options
            states = ['All States'] + sorted(self.stats_data['state'].unique().tolist())
            counties = ['All Counties'] + sorted(self.stats_data['county'].unique().tolist())
            
            metrics = [
                ('Population', 'population'),
                ('GDP (Millions)', 'gdp_millions'),
                ('GDP Per Capita', 'gdp_per_capita'),
                ('Median Income', 'median_income'),
                ('Unemployment Rate', 'unemployment_rate'),
                ('Poverty Rate', 'poverty_rate'),
                ('College Grad Rate', 'college_grad_rate'),
                ('Crime Rate', 'crime_rate_per_100k'),
                ('Life Expectancy', 'life_expectancy'),
                ('Median Home Value', 'median_home_value'),
                ('Broadband Access', 'broadband_access_pct'),
                ('Air Quality Index', 'air_quality_index')
            ]
            
            # Create widgets
            self.state_dropdown = widgets.Dropdown(
                options=states,
                value='All States', 
                description='State:',
                layout=widgets.Layout(width='180px')
            )
            
            self.county_dropdown = widgets.Dropdown(
                options=counties,
                value='All Counties',
                description='County:',
                layout=widgets.Layout(width='220px')
            )
            
            self.metric_dropdown = widgets.Dropdown(
                options=metrics,
                value='population',
                description='Metric:',
                layout=widgets.Layout(width='180px')
            )
            
            self.view_dropdown = widgets.Dropdown(
                options=[
                    ('Map View', 'map'),
                    ('Time Trends', 'trends'), 
                    ('Distribution', 'distribution'),
                    ('Comparison', 'comparison')
                ],
                value='map',
                description='View:',
                layout=widgets.Layout(width='150px')
            )
            
            # Update county list when state changes
            def update_counties(change):
                if change['new'] == 'All States':
                    new_counties = ['All Counties'] + sorted(self.stats_data['county'].unique().tolist())
                else:
                    state_counties = self.stats_data[self.stats_data['state'] == change['new']]['county'].unique()
                    new_counties = ['All Counties'] + sorted(state_counties.tolist())
                
                self.county_dropdown.options = new_counties
                self.county_dropdown.value = 'All Counties'
            
            self.state_dropdown.observe(update_counties, names='value')
            
            # Update dashboard when controls change
            def update_dashboard(*args):
                self.update_visualizations()
            
            for widget in [self.state_dropdown, self.county_dropdown, self.metric_dropdown, self.view_dropdown]:
                widget.observe(update_dashboard, names='value')
            
            # Controls layout
            controls_html = f'''
            <div style="
                background: {self.colors['surface']};
                padding: 25px;
                margin-bottom: 30px;
                border: 1px solid {self.colors['border']};
                border-radius: 8px;
            ">
                <h3 style="
                    margin: 0 0 20px 0;
                    color: {self.colors['text']};
                    font-size: 1.1rem;
                    font-weight: 500;
                ">Dashboard Controls</h3>
            </div>
            '''
            display(HTML(controls_html))
            
            # Widget container
            controls_container = widgets.HBox([
                self.state_dropdown,
                self.county_dropdown,
                self.metric_dropdown,
                self.view_dropdown
            ], layout=widgets.Layout(justify_content='space-around'))
            
            display(controls_container)
    
    def create_summary_statistics(self):
        """Create summary stats cards"""
        with self.stats_output:
            # Get latest data
            latest_data = self.stats_data[self.stats_data['date'] == self.stats_data['date'].max()]
            
            # Filter by state if selected
            if self.state_dropdown.value != 'All States':
                latest_data = latest_data[latest_data['state'] == self.state_dropdown.value]
            
            # Calculate stats
            total_pop = latest_data['population'].sum()
            avg_gdp_pc = latest_data['gdp_per_capita'].mean()
            avg_unemployment = latest_data['unemployment_rate'].mean()
            avg_income = latest_data['median_income'].mean()
            avg_education = latest_data['college_grad_rate'].mean()
            total_counties = len(latest_data)
            
            stats_html = f'''
            <div style="
                display: grid;
                grid-template-columns: repeat(auto-fit, minmax(180px, 1fr));
                gap: 20px;
                margin: 30px 0;
            ">
                <div style="
                    background: {self.colors['surface']};
                    padding: 24px;
                    border: 1px solid {self.colors['border']};
                    border-radius: 8px;
                    text-align: center;
                ">
                    <div style="font-size: 2rem; font-weight: 300; color: {self.colors['text']}; margin-bottom: 8px;">
                        {total_pop:,.0f}
                    </div>
                    <div style="font-size: 0.875rem; color: {self.colors['text_light']}; font-weight: 500;">
                        Total Population
                    </div>
                </div>
                
                <div style="
                    background: {self.colors['surface']};
                    padding: 24px;
                    border: 1px solid {self.colors['border']};
                    border-radius: 8px;
                    text-align: center;
                ">
                    <div style="font-size: 2rem; font-weight: 300; color: {self.colors['text']}; margin-bottom: 8px;">
                        ${avg_gdp_pc:,.0f}
                    </div>
                    <div style="font-size: 0.875rem; color: {self.colors['text_light']}; font-weight: 500;">
                        Avg GDP Per Capita
                    </div>
                </div>
                
                <div style="
                    background: {self.colors['surface']};
                    padding: 24px;
                    border: 1px solid {self.colors['border']};
                    border-radius: 8px;
                    text-align: center;
                ">
                    <div style="font-size: 2rem; font-weight: 300; color: {self.colors['text']}; margin-bottom: 8px;">
                        {avg_unemployment:.1f}%
                    </div>
                    <div style="font-size: 0.875rem; color: {self.colors['text_light']}; font-weight: 500;">
                        Avg Unemployment
                    </div>
                </div>
                
                <div style="
                    background: {self.colors['surface']};
                    padding: 24px;
                    border: 1px solid {self.colors['border']};
                    border-radius: 8px;
                    text-align: center;
                ">
                    <div style="font-size: 2rem; font-weight: 300; color: {self.colors['text']}; margin-bottom: 8px;">
                        ${avg_income:,.0f}
                    </div>
                    <div style="font-size: 0.875rem; color: {self.colors['text_light']}; font-weight: 500;">
                        Avg Median Income
                    </div>
                </div>
                
                <div style="
                    background: {self.colors['surface']};
                    padding: 24px;
                    border: 1px solid {self.colors['border']};
                    border-radius: 8px;
                    text-align: center;
                ">
                    <div style="font-size: 2rem; font-weight: 300; color: {self.colors['text']}; margin-bottom: 8px;">
                        {avg_education:.1f}%
                    </div>
                    <div style="font-size: 0.875rem; color: {self.colors['text_light']}; font-weight: 500;">
                        College Grad Rate
                    </div>
                </div>
                
                <div style="
                    background: {self.colors['surface']};
                    padding: 24px;
                    border: 1px solid {self.colors['border']};
                    border-radius: 8px;
                    text-align: center;
                ">
                    <div style="font-size: 2rem; font-weight: 300; color: {self.colors['text']}; margin-bottom: 8px;">
                        {total_counties}
                    </div>
                    <div style="font-size: 0.875rem; color: {self.colors['text_light']}; font-weight: 500;">
                        Counties
                    </div>
                </div>
            </div>
            '''
            display(HTML(stats_html))
    
    def create_map_visualization(self):
        """Create choropleth map"""
        # Get latest data
        latest_data = self.stats_data[self.stats_data['date'] == self.stats_data['date'].max()]
        
        # Filter by state if selected
        if self.state_dropdown.value != 'All States':
            latest_data = latest_data[latest_data['state'] == self.state_dropdown.value]
            map_gdf = self.gdf[self.gdf['STUSPS'] == self.state_dropdown.value].copy()
        else:
            map_gdf = self.gdf.copy()
        
        # Merge with boundaries
        map_data = map_gdf.merge(latest_data, left_on='NAME', right_on='county', how='left')
        
        # Create map
        if len(map_data) > 0:
            center_lat = map_data.geometry.centroid.y.mean()
            center_lon = map_data.geometry.centroid.x.mean()
            zoom_start = 6 if self.state_dropdown.value != 'All States' else 4
            
            m = folium.Map(
                location=[center_lat, center_lon],
                zoom_start=zoom_start,
                tiles=None
            )
            
            # Add clean tiles
            folium.TileLayer(
                tiles='https://{s}.basemaps.cartocdn.com/light_nopoi/{z}/{x}/{y}{r}.png',
                attr='© CARTO',
                name='Minimal',
                overlay=False,
                control=True
            ).add_to(m)
            
            # Add choropleth
            metric = self.metric_dropdown.value
            if metric in map_data.columns and not map_data[metric].isna().all():
                folium.Choropleth(
                    geo_data=map_data.__geo_interface__,
                    name=metric,
                    data=latest_data,
                    columns=['county', metric],
                    key_on='feature.properties.NAME',
                    fill_color='Greys',
                    fill_opacity=0.7,
                    line_opacity=0.3,
                    line_color='#718096',
                    line_weight=0.5,
                    legend_name=dict(self.metric_dropdown.options)[metric]
                ).add_to(m)
                
                # Add tooltips
                for idx, row in map_data.iterrows():
                    if pd.notna(row.get(metric)):
                        popup_html = f'''
                        <div style="
                            padding: 16px;
                            font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', sans-serif;
                            min-width: 200px;
                        ">
                            <h4 style="margin: 0 0 12px 0; color: #2D3748;">{row['NAME']}</h4>
                            <div style="color: #4A5568; font-size: 0.9rem;">
                                <strong>{dict(self.metric_dropdown.options)[metric]}:</strong><br>
                                {row[metric]:,.1f}
                            </div>
                        </div>
                        '''
                        
                        folium.GeoJson(
                            row.geometry.__geo_interface__,
                            popup=folium.Popup(popup_html, max_width=300),
                            tooltip=row['NAME'],
                            style_function=lambda x: {
                                'fillOpacity': 0,
                                'weight': 1,
                                'color': '#718096'
                            }
                        ).add_to(m)
            
            return m
        else:
            # Empty map
            return folium.Map(location=[39.8, -98.5], zoom_start=4)
    
    def create_chart_visualization(self):
        """Create chart visualizations"""
        view_type = self.view_dropdown.value
        metric = self.metric_dropdown.value
        
        # Filter data
        data = self.stats_data.copy()
        
        if self.state_dropdown.value != 'All States':
            data = data[data['state'] == self.state_dropdown.value]
        
        # Minimal chart styling
        chart_layout = dict(
            font=dict(family="-apple-system, BlinkMacSystemFont, 'Segoe UI', sans-serif", color='#2D3748'),
            plot_bgcolor='rgba(0,0,0,0)',
            paper_bgcolor='rgba(0,0,0,0)',
            margin=dict(l=50, r=40, t=60, b=40),
            showlegend=False,
            height=500
        )
        
        if view_type == 'trends':
            # Time series
            if self.county_dropdown.value != 'All Counties':
                trend_data = data[data['county'] == self.county_dropdown.value]
                title = f"{dict(self.metric_dropdown.options)[metric]} - {self.county_dropdown.value}"
            else:
                trend_data = data.groupby('date')[metric].mean().reset_index()
                title = f"Average {dict(self.metric_dropdown.options)[metric]}"
            
            fig = go.Figure()
            fig.add_trace(go.Scatter(
                x=trend_data['date'],
                y=trend_data[metric],
                mode='lines+markers',
                line=dict(color='#4A5568', width=2),
                marker=dict(size=4, color='#2D3748')
            ))
            
        elif view_type == 'distribution':
            # Histogram
            latest_data = data[data['date'] == data['date'].max()]
            
            fig = go.Figure()
            fig.add_trace(go.Histogram(
                x=latest_data[metric],
                nbinsx=15,
                marker=dict(color='#718096', opacity=0.7)
            ))
            title = f"Distribution of {dict(self.metric_dropdown.options)[metric]}"
            
        elif view_type == 'comparison':
            # Bar chart
            latest_data = data[data['date'] == data['date'].max()]
            top_data = latest_data.nlargest(12, metric)
            
            fig = go.Figure()
            fig.add_trace(go.Bar(
                x=top_data[metric],
                y=top_data['county'],
                orientation='h',
                marker=dict(color='#4A5568')
            ))
            title = f"Top Counties - {dict(self.metric_dropdown.options)[metric]}"
        
        fig.update_layout(
            title=dict(text=title, font=dict(size=16)),
            **chart_layout
        )
        
        return fig
    
    def update_visualizations(self):
        """Update all visualizations"""
        with self.map_output:
            clear_output(wait=True)
            
            if self.view_dropdown.value == 'map':
                map_obj = self.create_map_visualization()
                display(map_obj._repr_html_())
            else:
                fig = self.create_chart_visualization()
                fig.show()
        
        # Update summary stats
        self.create_summary_statistics()
    
    def launch_dashboard(self):
        """Launch the complete dashboard"""
        print('\n🚀 Launching US Counties Analytics Dashboard')
        print('=' * 50)
        
        # Load data
        print('\n📂 Loading county boundaries...')
        self.download_counties_data()
        
        print('\n📊 Generating statistics...')
        self.generate_county_statistics()
        
        print('\n🎨 Building interface...')
        self.create_dashboard_header()
        self.create_interactive_controls()
        self.create_summary_statistics()
        
        print('\n🗺️ Rendering initial visualization...')
        with self.map_output:
            self.update_visualizations()
        
        # Apply clean styling
        style_html = '''
        <style>
        .widget-dropdown select {
            border: 1px solid #E2E8F0;
            border-radius: 6px;
            padding: 10px 12px;
            background: #FFFFFF;
            color: #1A202C;
            font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', sans-serif;
            font-size: 0.875rem;
        }
        .widget-dropdown select:focus {
            outline: none;
            border-color: #718096;
            box-shadow: 0 0 0 3px rgba(113,128,150,0.1);
        }
        .widget-hbox {
            display: flex;
            gap: 25px;
            align-items: flex-end;
            flex-wrap: wrap;
            justify-content: space-around;
        }
        </style>
        '''
        display(HTML(style_html))
        
        # Create main layout
        dashboard_layout = widgets.VBox([
            self.header_output,
            self.controls_output,
            self.stats_output,
            self.map_output
        ])
        
        display(dashboard_layout)
        
        print('\n✅ Dashboard launched successfully!')
        print('🎯 Use the controls above to explore county data')
        print('🗺️ Try different states, metrics, and visualization types')

# Initialize dashboard
dashboard = SimpleCountiesDashboard()
print('Dashboard class initialized ✅')

In [ ]:
# Launch the dashboard
dashboard.launch_dashboard()